In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold, train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score, f1_score, precision_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

In [3]:
datasets = {
    'df_cds_motifs': pd.read_csv('data/processed/df_cds_motifs.csv'),
    'df_cds_smcog': pd.read_csv('data/processed/df_cds_smcog.csv'),
    'df_pfam_desc': pd.read_csv('data/processed/df_pfam_desc.csv'),
    'df_pfam_go': pd.read_csv('data/processed/df_pfam_go.csv'),
    'df_pfam_id': pd.read_csv('data/processed/df_pfam_id.csv')
}

for key, df in datasets.items():
    print(df.shape)

(266, 87)
(266, 243)
(266, 1558)
(266, 466)
(266, 1654)


Antifungi

In [6]:
df_merged = pd.DataFrame()
for key, df in datasets.items():
    df_merged = pd.concat([df_merged, df], axis=1)

# Check for duplicate column names
duplicated_columns = df_merged.columns[df_merged.columns.duplicated()]

# Drop columns with duplicate names
df_merged = df_merged.loc[:, ~df_merged.columns.duplicated()]

df_merged = df_merged.reset_index(drop=True)

steps = [('scaler', StandardScaler()), ('model', SVC(C = 10, gamma = 0.0001, kernel = 'rbf', probability = True, random_state = 0))]
model = Pipeline(steps = steps)

# df_merged = datasets['df_pfam_id']

y = df_merged.pop('label')
df_merged.pop('nome')
df_merged.pop('acc')
X = df_merged

scores_acc = cross_val_score(model, X, y, cv = 10, scoring = 'accuracy', n_jobs = -1)
scores_auc = cross_val_score(model, X, y, cv = 10, scoring = 'roc_auc', n_jobs = -1)

print(y.value_counts())
print('ACC: {:.4f}, AUC: {:.4f}'.format(scores_acc.mean(), scores_auc.mean()))

label
0    190
1     76
Name: count, dtype: int64
ACC: 0.7141, AUC: 0.7488
